In [1]:
import sys,string,os,math,shutil,time,glob
import numpy as np
from astropy.io import fits
from scipy.optimize import shgo, minimize
from scipy import optimize, special
from scipy.stats import norm, chisquare
from time import time
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from collections import Counter
import pandas as pd
import convol
import utils
import csv
import random
from scipy import ndimage
from PIL import Image

In [ ]:
ruta = 'T24500/'
if os.path.exists(ruta) and os.path.isdir(ruta):
    elementos = os.listdir(ruta)
    cantidad_elementos = len(elementos)
    print(f"La carpeta '{ruta}' contiene {cantidad_elementos} modelos.")
else:
    print(f"La ruta '{ruta}' no existe.")


In [ ]:
vsini = [i for i in range(30, 301, 10)]
len (vsini)

# Creación base de datos sin ruido 

In [ ]:
folder = "Modelos 6500/Parte 1/"
lineas_usar = ['SiIII4552']
resol = 46000
vsini = [i for i in range(30, 301, 15)]
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['axes.facecolor'] = 'white'

def generate_profiles():
    file_list = os.listdir(folder)
    for file_name in file_list:
        if file_name.endswith('.fits'):
            star_fit = folder + file_name
            hdulist = fits.open(star_fit)
            tbdata = hdulist[1].data
            cols = hdulist[1].columns
            names = cols.names
            for index, line in enumerate(names):
                ln = line.split('_')[0]
                if ln in lineas_usar and line.split('_')[1] == 'VT001':
                    vmicro = float(line.split('_')[1][2:7])
                    profile = tbdata[cols.names[index]]
                    sx = profile[:, 0]
                    sy = profile[:, 1]
                    cx_values = []
                    cy_values = []
                    for vm in vsini:
                        cx, cy = convol.convol(sx, sy, vm, 0, 0, resol)
                        cx_values.append(cx)
                        cy_values.append(cy)
                    for i, (cx, cy) in enumerate(zip(cx_values, cy_values)):
                        label = file_name.split('.')[0].split(' ')[0]
                        num = i + 1
                        filename = f"{label}_46_{num:02d}.jpg"
                        profile_data = {
                            'cx': cx,
                            'cy': cy,
                            'label': label,
                            'vsini': vsini[i],
                            'filename': filename
                        }
                        save_profile_as_image(profile_data)
                        save_vft_to_csv(profile_data)

def save_profile_as_image(profile_data):
    cx = profile_data['cx']
    cy = profile_data['cy']
    cx_exp = np.arange(4535, 4565, (cx[1] - cx[0]))
    cy_exp = np.interp(cx_exp, cx, cy)
    label = profile_data['label']
    filename = f"DATABASE/Perfiles rotados (sin ruido)/{profile_data['filename']}"
    plt.figure(figsize=(4, 4), dpi=100)
    plt.plot(cx_exp, cy_exp, linewidth=1.5)
    plt.grid(True)
    plt.ylim(0.6, 1.1)
    plt.savefig(filename)
    plt.close()
    image = Image.open(filename)
    resized_image = image.resize((400, 400)).convert('L')
    resized_image.save(filename, format='JPEG')

def save_vft_to_csv(profile_data):
    label = profile_data['label']
    vft = "{:.3f}".format(profile_data['vsini'])
    folder = "DATABASE/vft values (sin ruido)/"
    os.makedirs(folder, exist_ok=True)
    filename = os.path.join(folder, "vft_values.csv")
    if not os.path.exists(filename):
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Modelos', 'vft values'])
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([profile_data['filename'], vft])

generate_profiles()


## Modelos sintéticos para artículo (modelos .dat)

In [7]:
folder_path = "Modelos articulo/Validación/"
model_files = os.listdir(folder_path)

for model_file in model_files:
    model = os.path.join(folder_path, model_file)
    w,f = utils.read_asc(model)
    file_path ='iacob_broad_lines.dat'
    line_list, lambda_list = np.genfromtxt(file_path, dtype=None, unpack=True, usecols=(0,1),encoding=None)
    line = 'SiIII'
    if 'line' in locals() or 'line' in globals():
        if line:
            ll = np.where(line_list == line)[0]
            lam0 = lambda_list[ll[0]]  
            xsh = 20.
            xr0 = np.array([lam0 - xsh, lam0 + xsh])
            line_lab = '_' + line
        else:
            line = ''
    tt = np.where((w >= 4551) & (w <= 4554))
    w = w[tt]
    f = f[tt]
    resol = 46000
    vsini = [i for i in range(10, 301, 5)]
    mpl.rcParams['figure.facecolor'] = 'white'
    mpl.rcParams['axes.facecolor'] = 'white'

    def generate_profiles():
        cx_values = []
        cy_values = []
        sx = w
        sy = f
        for vm in vsini:
            cx, cy = convol.convol(sx, sy, vm, 0, 0, resol)
            cx_values.append(cx)
            cy_values.append(cy)
        for i, (cx, cy) in enumerate(zip(cx_values, cy_values)):
            label = model_file
            num = i + 1
            filename = f"{label}_{num:02d}.jpg"
            profile_data = {
                'cx': cx,
                'cy': cy,
                'label': label,
                'vsini': vsini[i],
                'filename': filename
            }
            save_profile_as_image(profile_data)
            save_vft_to_csv(profile_data)

    def save_profile_as_image(profile_data):
        cx = profile_data['cx']
        cy = profile_data['cy']
        cx_exp = np.arange(4550, 4555, (cx[1] - cx[0]))
        cy_exp = np.interp(cx_exp, cx, cy)
        #noise = 0.003 * np.random.randn(len(cy_exp))  ### Línea agregada para noise
        #cy_exp += noise                              ### Línea agregada para noise
        label = profile_data['label']
        filename = f"DATABASE/Validación (RESNET 6)/Perfiles rotados/{profile_data['filename']}"
        plt.figure(figsize=(4, 4), dpi=100)
        plt.plot(cx_exp, cy_exp, linewidth=1.5)  
        plt.grid(True)
        plt.ylim(0.6, 1.1)  
        plt.savefig(filename)
        plt.close()
        image = Image.open(filename)
        resized_image = image.resize((400, 400)).convert('L')
        resized_image.save(filename, format='JPEG')
             
    def save_vft_to_csv(profile_data):
        label = profile_data['label']
        vft = "{:.3f}".format(profile_data['vsini'])
        folder = "DATABASE/Validación (RESNET 6)/vft values/"
        os.makedirs(folder, exist_ok=True)
        filename = os.path.join(folder, "vft_values.csv")
        if not os.path.exists(filename):
            with open(filename, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['Modelos', 'vft values'])
        with open(filename, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([profile_data['filename'], vft])

    generate_profiles()

# Creación base de datos con ruido (modelos .fits)

In [ ]:
folder = "Modelos articulo"
lineas_usar = ['SiIII4552']
resol = 46000
vsini = [i for i in range(10, 301, 5)]
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['axes.facecolor'] = 'white'

def generate_profiles():
    file_list = os.listdir(folder)
    for file_name in file_list:
        if file_name.endswith('.fits'):
            star_fit = folder + file_name
            hdulist = fits.open(star_fit)
            tbdata = hdulist[1].data
            cols = hdulist[1].columns
            names = cols.names
            for index, line in enumerate(names):
                ln = line.split('_')[0]
                if ln in lineas_usar and line.split('_')[1] == 'VT001':
                    vmicro = float(line.split('_')[1][2:7])
                    profile = tbdata[cols.names[index]]
                    sx = profile[:, 0]
                    sy = profile[:, 1]
                    cx_values = []
                    cy_values = []
                    for vm in vsini:
                        cx, cy = convol.convol(sx, sy, vm, 0, 0, resol)
                        cx_values.append(cx)
                        cy_values.append(cy)
                    for i, (cx, cy) in enumerate(zip(cx_values, cy_values)):
                        label = file_name.split('.')[0].split(' ')[0]
                        num = i + 1
                        filename = f"{label}_r_{num:02d}.jpg"
                        profile_data = {
                            'cx': cx,
                            'cy': cy,
                            'label': label,
                            'vsini': vsini[i],
                            'filename': filename
                        }
                        save_profile_as_image(profile_data)
                        save_vft_to_csv(profile_data)

def save_profile_as_image(profile_data):
    cx = profile_data['cx']
    cy = profile_data['cy']
    cx_exp = np.arange(4535, 4565, (cx[1] - cx[0]))
    cy_exp = np.interp(cx_exp, cx, cy)
    noise = 0.01 * np.random.randn(len(cy_exp))
    cy_exp += noise
    label = profile_data['label']
    filename = f"DATABASE/Validación (RESNET 4)/Perfiles rotados/{profile_data['filename']}"
    plt.figure(figsize=(4, 4), dpi=100)
    plt.plot(cx_exp, cy_exp, linewidth=1.5)
    plt.grid(True)
    plt.ylim(0.6, 1.1)
    plt.savefig(filename)
    plt.close()
    image = Image.open(filename)
    resized_image = image.resize((400, 400)).convert('L')
    resized_image.save(filename, format='JPEG')

def save_vft_to_csv(profile_data):
    label = profile_data['label']
    vft = "{:.3f}".format(profile_data['vsini'])
    folder = "DATABASE/Validación (RESNET 4)/vft values/"
    os.makedirs(folder, exist_ok=True)
    filename = os.path.join(folder, "vft_values.csv")
    if not os.path.exists(filename):
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Modelos', 'vft values'])
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([profile_data['filename'], vft])

generate_profiles()

In [ ]:
2+2

In [ ]:
def combine_csv_files(file1, file2, output_file):
    with open(file1, 'r') as csv_file1, open(file2, 'r') as csv_file2, open(output_file, 'w', newline='') as output_csv:
        reader1 = csv.reader(csv_file1)
        reader2 = csv.reader(csv_file2)
        writer = csv.writer(output_csv)
        header1 = next(reader1)
        writer.writerow(header1)
        for row in reader1:
            writer.writerow(row)
        header2 = next(reader2)
        for row in reader2:
            writer.writerow(row)

file1 = "DATABASE/Perfiles RESNET 3/vft values/vft_values.csv"
file2 = "DATABASE/Perfiles RESNET 4/vft values/vft_values.csv"
output_file = "DATABASE/vft_values.csv"
combine_csv_files(file1, file2, output_file)

## FFT de modelos 

In [ ]:
dirin       = "Modelos articulo/"
model       = "t20000g35-interp"
w, f = utils.read_asc(dirin+model+".dat")
file_path ='iacob_broad_lines.dat'
line_list, lambda_list = np.genfromtxt(file_path, dtype=None, unpack=True, usecols=(0,1),encoding=None)
line = 'SiIII'
if 'line' in locals() or 'line' in globals():
    if line:
        ll = np.where(line_list == line)[0]
        lam0 = lambda_list[ll[0]]  
        xsh = 20.
        xr0 = np.array([lam0 - xsh, lam0 + xsh])
        line_lab = '_' + line
    else:
        line = ''
tt = np.where((w >= xr0[0]) & (w <= xr0[1]))
w = w[tt]
f = f[tt]
resol = 46000
vsini = [30,100,200]
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['axes.facecolor'] = 'white'

def generate_profiles():
    profiles = []  
    cx_values = []
    cy_values = []
    sx = w
    sy = f
    for vm in vsini:
        cx, cy = convol.convol(sx, sy, vm, 0, 0, resol)
        cx_values.append(cx)
        cy_values.append(cy)
    for i, (cx, cy) in enumerate(zip(cx_values, cy_values)):
        label = model
        num = i + 1
        filename = f"{label}_{num:02d}.jpg"
        profile_data = {
            'cx': cx,
            'cy': cy,
            'label': label,
            'vsini': vsini[i],
            'filename': filename
        }
        profiles.append(profile_data) 

    w1, f1 = profiles[0]['cx'], profiles[0]['cy']
    w2, f2 = profiles[1]['cx'], profiles[1]['cy']
    w3, f3 = profiles[2]['cx'], profiles[2]['cy']
    return w1, f1, w2, f2, w3, f3

w1, f1, w2, f2, w3, f3 = generate_profiles()
xmin = 4552.3
xmax = 4552.82
ii = np.where((w1>xmin) & (w1<xmax))
w1,f1,w2,f2,w3,f3 = w1[ii],f1[ii],w2[ii],f2[ii],w3[ii],f3[ii]

plt.plot(w1, f1)
plt.title(model+' 30$[km/h]$')
plt.show()
plt.plot(w2, f2)
plt.title(model+' 100$[km/h]$')
plt.show()
plt.plot(w3, f3)
plt.title(model+' 200$[km/h]$')
plt.show()

In [ ]:
#========== FT perfil 1 =================================#
xc = lam0
xfou0, yfou0, max_sigma = utils.vfft(w1, f1, xc)
y_0 = -1*yfou0
peaks = find_peaks(y_0)
peaks_pos = np.array(xfou0[peaks[0]])
vft = peaks_pos[0]
peaks_height = -1*y_0[peaks[0]]
vft_y = peaks_height[0]

#=======================================================================#
plt.figure(figsize=(10,10))
plt.plot(xfou0, yfou0,color='red',lw=1,ls='--',label='FT perfil observado reducido')
#plt.plot(xfou1, yfou1,color='blue',lw=1,ls='--',label='FT ajuste gaussiano reducido')
plt.axis([0,vft*2,-4,0])
plt.text(max_sigma, -0.6, 'Nyq.', fontsize=10, ha='right', va='bottom', rotation=90)
plt.text(vft, vft_y-0.3, r'{:.2f} [$\frac{{km}}{{s}}$]'.format(vft), fontsize=14)
#plt.text(vft1, vft_y1-0.3, r'{:.2f} [$\frac{{km}}{{s}}$]'.format(vft1), fontsize=14)
plt.axvline(vft,color='black',linewidth=1.5,linestyle='--')
#plt.axvline(vft1,color='black',linewidth=1.5,linestyle='--')
plt.scatter(peaks_pos,peaks_height,color='darkmagenta',s=60,marker='X',label='Peaks perfil observado')
#plt.scatter(peaks_pos1,peaks_height1,color='cyan',s=60,marker='o',label='Peaks perfil gaussiano')
plt.xlabel('v sini (km s$^{-1}$)')
plt.ylabel('ABS (AMPLITUDE)')
plt.title('Fourier transform '+ model,fontsize=14)
plt.legend()
plt.grid()
plt.show()